In [1]:
names = [
    "Роман",
    "Детектив",
    "Фантастика",
    "Приключения",
    "Ужасы (Хоррор)",
    "Фэнтези",
    "Исторический роман",
    "Триллер",
    "Драма",
    "Поэзия",
    "Научно-популярная литература",
    "Биография/автобиография",
    "Философия",
    "Психология",
    "Эссе",
    "Классика",
    "Зарубежная классика",
    "Публицистика",
    "Фольклор",
    "Комиксы/графические романы",
    "Религиозная литература"
]

In [25]:
import requests
from lxml import html
import tqdm 

links = []

for name in names:
    links.append(f'https://www.labirint.ru/search/{name}/?stype=1&page=')

all_links = []

for link in links:
    for i in tqdm.tqdm(range(18)):
        response = requests.get(link + str(i))

        html_content = response.text

        tree = html.fromstring(html_content)

        product_names = tree.xpath('//a[@class="product-card__name"]')

        for product_name in product_names:
            href = product_name.get('href')
            all_links.append('https://www.labirint.ru' + href)

    print(len(all_links))

100%|██████████| 18/18 [00:05<00:00,  3.59it/s]


0


100%|██████████| 18/18 [00:12<00:00,  1.39it/s]


1060


100%|██████████| 18/18 [00:12<00:00,  1.39it/s]


...

14332


100%|██████████| 18/18 [00:05<00:00,  3.47it/s]


14332


100%|██████████| 18/18 [00:15<00:00,  1.13it/s]

15392


In [1]:
import pandas as pd

pd_links = pd.DataFrame({'link': all_links}).drop_duplicates()
pd_links.to_csv('links_to_books.csv', index=False)

NameError: name 'all_links' is not defined

In [94]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-giant')
model = AutoModel.from_pretrained('facebook/dinov2-giant')

def embed_picture(image):
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.detach().numpy()[0].flatten()
    return  embedding

In [1]:
import pandas as pd
import requests
from lxml import html
import tqdm 
import re
from PIL import Image

pd_links = pd.read_csv('links_to_books.csv')
pd_links.head()

,link
0,https://www.labirint.ru/books/451006/
1,https://www.labirint.ru/books/786953/
2,https://www.labirint.ru/books/840898/
3,https://www.labirint.ru/books/614463/
4,https://www.labirint.ru/books/869738/


In [2]:
import logging

logging.basicConfig(
        level=logging.INFO,
        filename='parser.log',
        filemode="w",
        format="%(asctime)s %(levelname)s %(message)s",
        force=True,
    )

In [3]:
d = {'id': [], 'link': [], 'name': [], 'rate': [], 'description': [], 'reviews': []} #, 'embeddings': []}

for index, row in tqdm.tqdm(pd_links[11_500:].iterrows(), total=len(pd_links[11_500:])):
    try:
        if index % 500 == 0:
            books = pd.DataFrame(d) 
            books.to_csv('rows11_500-all.csv', index=False)

        book_id = row.link.split('/')[-2]
        response = requests.get(row.link, timeout=15)

        html_content = response.text
        tree = html.fromstring(html_content)

        name = tree.xpath("//div[@id='product-title']/h1")[0].text
        
        rate = tree.xpath("//div[@id='rate']")[0].text

        descr = []
        ps = tree.xpath("//div[@id='fullannotation']/p")
        if not ps:
            ps = tree.xpath("//div[@id='fullannotation']/noindex/p")
        for p in ps:
            descr.append(re.sub(r'\xa0', ' ', p.text_content()))

        descr = ' '.join(descr)

        if not descr:
            descr = tree.xpath("//div[@id='product-about']/p/noindex")
            if not descr:
                descr = tree.xpath("//div[@id='product-about']/p")
            descr = descr[0].text_content()

        reviews = []

        for div in tree.xpath("//div[contains(@class, 'product-comment')]/div[4]/div/div[2]/noindex"):
            reviews.append(div.text_content())

        for div in tree.xpath("//div[@class='comment-text content-comments']/div/p"):
            reviews.append(div.text_content())

        reviews = '\n;\n'.join(reviews)

        pic_url = tree.xpath("//div[@id='product-image']/img")[0].get("data-src")

        # with open(f'pics/{book_id}.jpg', 'wb') as f:
        #     response.raw.decode_content = True
        #     shutil.copyfileobj(response.raw, f)

        img = Image.open(requests.get(pic_url, stream=True).raw)
        img.save(f'pics/{book_id}.png')

        # embeddings = embed_picture(img)

        d['id'].append(book_id)
        d['link'].append(row.link)
        d['name'].append(name)
        d['rate'].append(rate)
        d['description'].append(descr)
        d['reviews'].append(reviews)
        
        #d['embeddings'].append(embeddings)
    except Exception:
        logging.exception(f"Exception occurred for link: {row.link}")

books = pd.DataFrame(d) 
books.to_csv('rows11_500-all.csv', index=False)

  0%|          | 0/1186 [00:00<?, ?it/s]

100%|██████████| 1186/1186 [27:11<00:00,  1.38s/it]


In [4]:
books = pd.DataFrame(d) 
books.to_csv('rows9_000-11_500.csv', index=False)

In [11]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By

# driver = webdriver.Firefox()

# d = {'id': [], 'link': [], 'name': [], 'rate': [], 'description': [], 'reviews': []} #, 'embeddings': []}

# for index, row in tqdm.tqdm(pd_links[2_000:4_000].iterrows(), total=pd_links[2_000:4_000].shape[0]):
#     try:
#         book_id = row.link.split('/')[-2]
        
#         driver.get(row.link)

#         name = driver.find_elements(By.XPATH, "//div[@id='product-title']/h1")[0].get_text()
        
#         rate = driver.find_elements(By.XPATH,"//div[@id='rate']")[0].get_text()

#         descr = []
#         ps = driver.find_elements(By.XPATH,"//div[@id='fullannotation']/p")
#         if not ps:
#             ps = driver.find_elements(By.XPATH,"//div[@id='fullannotation']/noindex/p")
#         for p in ps:
#             descr.append(re.sub(r'\xa0', ' ', p.get_text()))

#         descr = ' '.join(descr)

#         if not descr:
#             descr = driver.find_elements(By.XPATH,"//div[@id='product-about']/p/noindex")
#             if not descr:
#                 descr = driver.find_elements(By.XPATH,"//div[@id='product-about']/p")
#             descr = descr[0].get_text()

#         reviews = []

#         for div in driver.find_elements(By.XPATH,"//div[contains(@class, 'product-comment')]/div[4]/div/div[2]/noindex"):
#             reviews.append(div.get_text())

#         for div in driver.find_elements(By.XPATH,"//div[@class='comment-text content-comments']/div/p"):
#             reviews.append(div.get_text())

#         reviews = '\n;\n'.join(reviews)

#         pic_url = driver.find_elements(By.XPATH,"//div[@id='product-image']/img")[0].get("data-src")

#         #response = requests.get(pic_url)

#         # with open(f'pics/{book_id}.jpg', 'wb') as f:
#         #     response.raw.decode_content = True
#         #     shutil.copyfileobj(response.raw, f)

#         # img = Image.open(requests.get(pic_url, stream=True).raw)
#         # img.save(f'pics/{book_id}.png')

#         # embeddings = embed_picture(img)

#         d['id'].append(book_id)
#         d['link'].append(row.link)
#         d['name'].append(name)
#         d['rate'].append(rate)
#         d['description'].append(descr)
#         d['reviews'].append(reviews)
        
#         #d['embeddings'].append(embeddings)
#     except Exception:
#         logging.exception(f"Exception occurred for link: {row.link}")

  1%|          | 13/2000 [00:24<56:33,  1.71s/it] 

In [ ]:
books = pd.DataFrame(d) 


In [ ]:
books.to_csv('rows7_000-9_000.csv', index=False)

In [10]:
d = {'id': [], 'link': [], 'name': [], 'rate': [], 'description': [], 'reviews': []} #, 'embeddings': []}

for index, row in tqdm.tqdm(pd_links[6_000:8_000].iterrows(), total=pd_links[6_000:8_000].shape[0]):
    try:
        book_id = row.link.split('/')[-2]
        response = requests.get(row.link)

        html_content = response.text
        tree = html.fromstring(html_content)

        name = tree.xpath("//div[@id='product-title']/h1")[0].text
        
        rate = tree.xpath("//div[@id='rate']")[0].text

        descr = []
        ps = tree.xpath("//div[@id='fullannotation']/p")
        if not ps:
            ps = tree.xpath("//div[@id='fullannotation']/noindex/p")
        for p in ps:
            descr.append(re.sub(r'\xa0', ' ', p.text_content()))

        descr = ' '.join(descr)

        if not descr:
            descr = tree.xpath("//div[@id='product-about']/p/noindex")
            if not descr:
                descr = tree.xpath("//div[@id='product-about']/p")
            descr = descr[0].text_content()

        reviews = []

        for div in tree.xpath("//div[contains(@class, 'product-comment')]/div[4]/div/div[2]/noindex"):
            reviews.append(div.text_content())

        for div in tree.xpath("//div[@class='comment-text content-comments']/div/p"):
            reviews.append(div.text_content())

        reviews = '\n;\n'.join(reviews)

        pic_url = tree.xpath("//div[@id='product-image']/img")[0].get("data-src")

        response = requests.get(pic_url)

        # with open(f'pics/{book_id}.jpg', 'wb') as f:
        #     response.raw.decode_content = True
        #     shutil.copyfileobj(response.raw, f)

        # img = Image.open(requests.get(pic_url, stream=True).raw)
        # img.save(f'pics/{book_id}.png')

        # embeddings = embed_picture(img)

        d['id'].append(book_id)
        d['link'].append(row.link)
        d['name'].append(name)
        d['rate'].append(rate)
        d['description'].append(descr)
        d['reviews'].append(reviews)
        
        #d['embeddings'].append(embeddings)
    except Exception:
        logging.exception(f"Exception occurred for link: {row.link}")

 87%|████████▋ | 1731/2000 [35:38<05:18,  1.19s/it]

In [ ]:
books = pd.DataFrame(d) 
books.to_csv('rows6_000-8_000.csv', index=False)

In [ ]:
d = {'id': [], 'link': [], 'name': [], 'rate': [], 'description': [], 'reviews': []} #, 'embeddings': []}

for index, row in tqdm.tqdm(pd_links[8_000:10_000].iterrows(), total=pd_links[8_000:10_000].shape[0]):
    try:
        book_id = row.link.split('/')[-2]
        response = requests.get(row.link)

        html_content = response.text
        tree = html.fromstring(html_content)

        name = tree.xpath("//div[@id='product-title']/h1")[0].text
        
        rate = tree.xpath("//div[@id='rate']")[0].text

        descr = []
        ps = tree.xpath("//div[@id='fullannotation']/p")
        if not ps:
            ps = tree.xpath("//div[@id='fullannotation']/noindex/p")
        for p in ps:
            descr.append(re.sub(r'\xa0', ' ', p.text_content()))

        descr = ' '.join(descr)

        if not descr:
            descr = tree.xpath("//div[@id='product-about']/p/noindex")
            if not descr:
                descr = tree.xpath("//div[@id='product-about']/p")
            descr = descr[0].text_content()

        reviews = []

        for div in tree.xpath("//div[contains(@class, 'product-comment')]/div[4]/div/div[2]/noindex"):
            reviews.append(div.text_content())

        for div in tree.xpath("//div[@class='comment-text content-comments']/div/p"):
            reviews.append(div.text_content())

        reviews = '\n;\n'.join(reviews)

        pic_url = tree.xpath("//div[@id='product-image']/img")[0].get("data-src")

        response = requests.get(pic_url)

        # with open(f'pics/{book_id}.jpg', 'wb') as f:
        #     response.raw.decode_content = True
        #     shutil.copyfileobj(response.raw, f)

        # img = Image.open(requests.get(pic_url, stream=True).raw)
        # img.save(f'pics/{book_id}.png')

        # embeddings = embed_picture(img)

        d['id'].append(book_id)
        d['link'].append(row.link)
        d['name'].append(name)
        d['rate'].append(rate)
        d['description'].append(descr)
        d['reviews'].append(reviews)
        
        #d['embeddings'].append(embeddings)
    except Exception:
        logging.exception(f"Exception occurred for link: {row.link}")

In [ ]:
books = pd.DataFrame(d) 
books.to_csv('rows8_000-10_000.csv', index=False)

In [ ]:
d = {'id': [], 'link': [], 'name': [], 'rate': [], 'description': [], 'reviews': []} #, 'embeddings': []}

for index, row in tqdm.tqdm(pd_links[10_000:12_000].iterrows(), total=pd_links[10_000:12_000].shape[0]):
    try:
        book_id = row.link.split('/')[-2]
        response = requests.get(row.link)

        html_content = response.text
        tree = html.fromstring(html_content)

        name = tree.xpath("//div[@id='product-title']/h1")[0].text
        
        rate = tree.xpath("//div[@id='rate']")[0].text

        descr = []
        ps = tree.xpath("//div[@id='fullannotation']/p")
        if not ps:
            ps = tree.xpath("//div[@id='fullannotation']/noindex/p")
        for p in ps:
            descr.append(re.sub(r'\xa0', ' ', p.text_content()))

        descr = ' '.join(descr)

        if not descr:
            descr = tree.xpath("//div[@id='product-about']/p/noindex")
            if not descr:
                descr = tree.xpath("//div[@id='product-about']/p")
            descr = descr[0].text_content()

        reviews = []

        for div in tree.xpath("//div[contains(@class, 'product-comment')]/div[4]/div/div[2]/noindex"):
            reviews.append(div.text_content())

        for div in tree.xpath("//div[@class='comment-text content-comments']/div/p"):
            reviews.append(div.text_content())

        reviews = '\n;\n'.join(reviews)

        pic_url = tree.xpath("//div[@id='product-image']/img")[0].get("data-src")

        response = requests.get(pic_url)

        # with open(f'pics/{book_id}.jpg', 'wb') as f:
        #     response.raw.decode_content = True
        #     shutil.copyfileobj(response.raw, f)

        # img = Image.open(requests.get(pic_url, stream=True).raw)
        # img.save(f'pics/{book_id}.png')

        # embeddings = embed_picture(img)

        d['id'].append(book_id)
        d['link'].append(row.link)
        d['name'].append(name)
        d['rate'].append(rate)
        d['description'].append(descr)
        d['reviews'].append(reviews)
        
        #d['embeddings'].append(embeddings)
    except Exception:
        logging.exception(f"Exception occurred for link: {row.link}")

In [ ]:
books = pd.DataFrame(d) 
books.to_csv('rows10_000-12_000.csv', index=False)

In [101]:
pip install pytesseract

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [23]:
import cv2
import numpy as np
import pytesseract
from pytesseract import Output

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Load image, convert to HSV format, define lower/upper ranges, and perform
# color segmentation to create a binary mask
image = cv2.imread(r'pics\1008835.png')
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
lower = np.array([0, 0, 200])
upper = np.array([255, 255, 255])
mask = cv2.inRange(hsv, lower, upper)

# Create horizontal kernel and dilate to connect text characters
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,3))
dilate = cv2.dilate(mask, kernel, iterations=5)

# Find contours and filter using aspect ratio
# Remove non-text contours by filling in the contour
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    ar = w / float(h)
    if ar < 5:
        cv2.drawContours(dilate, [c], -1, (0,0,0), -1)

# Bitwise dilated image with mask, invert, then OCR
result = 255 - cv2.bitwise_and(dilate, mask)

#data = pytesseract.image_to_boxes(mask, lang='rus',config='--psm 6', output_type=Output.DICT)

cv2.imshow('mask', mask)
cv2.imshow('dilate', dilate)
cv2.imshow('result', result)
cv2.waitKey()

# n_boxes = len(data['char'])
# for i in range(n_boxes):
#     print(data['left'])
#     (x, y, w, h) = (data['left'][i], data['top'][i], data['right'][i] - data['left'][i], data['top'][i] - data['bottom'][i])

#     cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
#     print((x, y), (x + w, y + h))


# cv2.imshow('img', image)
# cv2.waitKey(0)

-1

In [7]:
import pytesseract
from pytesseract import Output
import cv2

# img = cv2.imread(r'pics\1010687.png')
# img = cv2.imread(r'pics\1010326.png')
img = cv2.imread(r'pics\1008835.png')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lower = np.array([0, 0, 100])
upper = np.array([255, 255, 255])
mask = cv2.inRange(hsv, lower, upper)

d = pytesseract.image_to_boxes(img, output_type=Output.DICT)
print(d)
n_boxes = len(d['level'])
for i in range(n_boxes):
    (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
    if d['conf'][i] > 0:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imshow('img', img)
cv2.waitKey(0)

{'char': ['~'], 'left': [0], 'bottom': [0], 'right': [360], 'top': [561], 'page': [0]}


KeyError: 'level'

In [25]:
d

{'level': [1,
  2,
  3,
  4,
  5,
  5,
  4,
  5,
  5,
  2,
  3,
  4,
  5,
  2,
  3,
  4,
  5,
  5,
  5,
  5,
  5,
  5,
  4,
  5,
  5,
  5,
  2,
  3,
  4,
  5],
 'page_num': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'block_num': [0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  4,
  4,
  4,
  4],
 'par_num': [0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1],
 'line_num': [0,
  0,
  0,
  1,
  1,
  1,
  2,
  2,
  2,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  0,
  0,
  1,
  1],
 'word_num': [0,
  0,
  0,
  0,
  1,
  2,
  0,
  1,
  2,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  0,
  1,
  2,
  3,
  0,
  0,
  0,
  1],
 'left': [0,
  82,
  82,


In [4]:
from PIL import Image

In [12]:
Image.fromarray(image).show()

In [14]:
Image.fromarray(mask).show()

In [11]:
Image.fromarray(dilate).show()

In [10]:
Image.fromarray(result).show()

In [5]:
from PIL import Image

In [3]:
result.shape

(542, 363)

In [3]:
cv2.imshow('result', result)

In [ ]:
1+ 2

3

: 